# Object Detection using TAO FasterRCNN

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080"> 

## Sample prediction of FasterRCNN
<img align="center" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/notebook/common/sample.jpg?raw=true" width="960"> 

 ## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 FasterRCNN model on the KITTI dataset
* Prune the trained FasterRCNN model
* Retrain the pruned model to recover lost accuracy
* Run evaluation & inference on the trained model to verify the accuracy
* Export & deploy the model in DeepStream/TensorRT
* Quantization-Aware Training(QAT) workflow for the best accuracy-performance trade-off

At the end of this notebook, you will have generated a trained and optimized `faster_rcnn` model
which you may deploy via [Triton](https://github.com/NVIDIA-AI-IOT/tao-toolkit-triton-apps)
or [DeepStream](https://developer.nvidia.com/deepstream-sdk).
 
 ### Table of Contents

 This notebook shows an example use case of FasterRCNN using Train Adapt Optimize (TAO) Toolkit.

 0. [Set up env variables and map drives](#head-0)
 1. [Install the TAO launcher](#head-1)
 2. [Prepare dataset and pretrained model](#head-2)<br>
     2.1 [Download the dataset](#head-2-1)<br>
     2.2 [Verify the downloaded dataset](#head-2-2)<br>
     2.3 [Prepare tfrecords from kitti format dataset](#head-2-3)<br>
     2.4 [Download pretrained model](#head-2-4)
 3. [Provide training specification](#head-3)
 4. [Run TAO training](#head-4)
 5. [Evaluate trained models](#head-5)
 6. [Prune trained models](#head-6)
 7. [Retrain pruned models](#head-7)
 8. [Evaluate retrained model](#head-8)
 9. [Visualize inferences](#head-9)
 10. [Deploy](#head-10)
 11. [QAT workflow](#head-11)<br>
     11.1 [Training](#head-11.1)<br>
     11.2 [Evaluation](#head-11.2)<br>
     11.3 [Pruning](#head-11.3)<br>
     11.4 [Retraining](#head-11.4)<br>
     11.5 [Evaluation of the retrained model](#head-11.5)<br>
     11.6 [Inference of the retrained model](#head-11.6)<br>
     11.7 [Deployment of the QAT model](#head-11.7)

 ## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>
 
The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

print("Please replace the variables with your own.")
%env GPU_INDEX=0

# Please define this local project directory that needs to be mapped to the TAO docker session.
%env LOCAL_PROJECT_DIR=/path/to/your/tao-experiments
os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "faster_rcnn"
)
%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/faster_rcnn
%env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data
# The sample spec files are present in the same path as the downloaded samples.
# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/faster_rcnn
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tao-experiments/faster_rcnn/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

In [ ]:
# Create local dir
!mkdir -p $LOCAL_DATA_DIR
!mkdir -p $LOCAL_EXPERIMENT_DIR

The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/text/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

```json
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    },
    # set gpu index for tao-converter
    "Envs": [
        {"variable": "CUDA_VISIBLE_DEVICES", "value": os.getenv("GPU_INDEX")},
    ]
}
```

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ],
    # set gpu index for tao-converter
    "Envs": [
        {"variable": "CUDA_VISIBLE_DEVICES", "value": os.getenv("GPU_INDEX")},
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Install the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 460+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

In [ ]:
# Skip this step if you have already installed the TAO launcher.
!pip3 install --upgrade nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info --verbose

 ## 2. Prepare dataset and pretrained model <a class="anchor" id="head-2"></a>

 We will be using the KITTI detection dataset for the tutorial. To find more details please visit
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d. Please download the KITTI detection images (http://www.cvlibs.net/download.php?file=data_object_image_2.zip) and labels (http://www.cvlibs.net/download.php?file=data_object_label_2.zip) to $DATA_DOWNLOAD_DIR.
 
 The data will then be extracted to have
 * training images in `$LOCAL_DATA_DIR/training/image_2`
 * training labels in `$LOCAL_DATA_DIR/training/label_2`
 * testing images in `$LOCAL_DATA_DIR/testing/image_2`
 
You may use this notebook with your own dataset as well. To use this example with your own dataset, please follow the same directory structure as mentioned below.

*Note: There are no labels for the testing images, therefore we use it just to visualize inferences for the trained model.*

### 2.1 Download the dataset <a class="anchor" id="head-2-1"></a>

Once you have gotten the download links in your email, please populate them in place of the `KITTI_IMAGES_DOWNLOAD_URL` and the `KITTI_LABELS_DOWNLOAD_URL`. This next cell, will download the data and place in `$LOCAL_DATA_DIR`

In [ ]:
import os
!mkdir -p $LOCAL_DATA_DIR
os.environ["URL_IMAGES"]=KITTI_IMAGES_DOWNLOAD_URL
!if [ ! -f $LOCAL_DATA_DIR/data_object_image_2.zip ]; then wget $URL_IMAGES -O $LOCAL_DATA_DIR/data_object_image_2.zip; else echo "image archive already downloaded"; fi 
os.environ["URL_LABELS"]=KITTI_LABELS_DOWNLOAD_URL
!if [ ! -f $LOCAL_DATA_DIR/data_object_label_2.zip ]; then wget $URL_LABELS -O $LOCAL_DATA_DIR/data_object_label_2.zip; else echo "label archive already downloaded"; fi 

### 2.2 Verify the downloaded dataset <a class="anchor" id="head-2-2"></a>

In [ ]:
# Check the dataset is present
!mkdir -p $LOCAL_DATA_DIR
!if [ ! -f $LOCAL_DATA_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $LOCAL_DATA_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi

In [ ]:
# This may take a while: verify integrity of zip files 
!sha256sum $LOCAL_DATA_DIR/data_object_image_2.zip | cut -d ' ' -f 1 | grep -xq '^351c5a2aa0cd9238b50174a3a62b846bc5855da256b82a196431d60ff8d43617$' ; \
if test $? -eq 0; then echo "images OK"; else echo "images corrupt, re-download!" && rm -f $LOCAL_DATA_DIR/data_object_image_2.zip; fi 
!sha256sum $LOCAL_DATA_DIR/data_object_label_2.zip | cut -d ' ' -f 1 | grep -xq '^4efc76220d867e1c31bb980bbf8cbc02599f02a9cb4350effa98dbb04aaed880$' ; \
if test $? -eq 0; then echo "labels OK"; else echo "labels corrupt, re-download!" && rm -f $LOCAL_DATA_DIR/data_object_label_2.zip; fi 

In [ ]:
# unpack 
!unzip -u $LOCAL_DATA_DIR/data_object_image_2.zip -d $LOCAL_DATA_DIR
!unzip -u $LOCAL_DATA_DIR/data_object_label_2.zip -d $LOCAL_DATA_DIR

In [ ]:
# verify
import os

DATA_DIR = os.environ.get('LOCAL_DATA_DIR')
num_training_images = len(os.listdir(os.path.join(DATA_DIR, "training/image_2")))
num_training_labels = len(os.listdir(os.path.join(DATA_DIR, "training/label_2")))
num_testing_images = len(os.listdir(os.path.join(DATA_DIR, "testing/image_2")))
print("Number of images in the train/val set. {}".format(num_training_images))
print("Number of labels in the train/val set. {}".format(num_training_labels))
print("Number of images in the test set. {}".format(num_testing_images))

### 2.3 Prepare tfrecords from kitti format dataset <a class="anchor" id="head-2-3"></a>

* Update the tfrecords spec file to take in your kitti format dataset
* Create the tfrecords using the dataset_convert 
* TFRecords only need to be generated once.

In [ ]:
print("TFrecords conversion spec file for training")
!cat $LOCAL_SPECS_DIR/frcnn_tfrecords_kitti_trainval.txt

In [ ]:
#KITTI trainval
!tao model faster_rcnn dataset_convert --gpu_index $GPU_INDEX -d $SPECS_DIR/frcnn_tfrecords_kitti_trainval.txt \
                     -o $DATA_DOWNLOAD_DIR/faster_rcnn/tfrecords/kitti_trainval/kitti_trainval \
                     -r $USER_EXPERIMENT_DIR/

In [ ]:
!ls -rlt $LOCAL_DATA_DIR/faster_rcnn/tfrecords/kitti_trainval

 ### 2.4 Download pre-trained model <a class="anchor" id="head-2-4"></a>

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!ngc registry model list nvidia/tao/pretrained_object_detection*

In [ ]:
# Download model from NGC.
!ngc registry model download-version nvidia/tao/pretrained_object_detection:resnet18

In [ ]:
# Copy weights to experiment directory.
!cp pretrained_object_detection_vresnet18/resnet_18.hdf5 $LOCAL_EXPERIMENT_DIR
!rm -rf pretrained_object_detection_vresnet18
!ls -rlt $LOCAL_EXPERIMENT_DIR

 ## 3. Provide training specification <a class="anchor" id="head-3"></a>

In [ ]:
!cat $LOCAL_SPECS_DIR/default_spec_resnet18.txt

 ## 4. Run TAO training <a class="anchor" id="head-4"></a>
 * Provide the sample spec file for training.

In [ ]:
!tao model faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt -r /workspace/tao-experiments/faster_rcnn

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -lht $LOCAL_EXPERIMENT_DIR

In [ ]:
print("For multi-GPU data parallelism, please uncomment and run this instead. Change --gpus  and --gpu_index based on your machine.")
# !tao model faster_rcnn train -e $SPECS_DIR/default_spec_resnet18.txt \
#                    --gpus 2 \
#                    --gpu_index 0 1 \
#                    -r /workspace/tao-experiments/faster_rcnn

In [ ]:
print("""
For multi-GPU model parallelism, please uncomment and run this instead.
Also add related parameters in training_config to enable model parallelism. E.g., 

             model_parallelism: 50
             model_parallelism: 50

""")

#!tao model faster_rcnn train -e $SPECS_DIR/default_spec_resnet18.txt \
#                   --gpus 2 \
#                   --gpu_index 0 1\
#                   -np 1 \
#                   -r /workspace/tao-experiments/faster_rcnn

In [ ]:
print("For resume training from checkpoint, please uncomment and run this instead. Change/Add the 'resume_from_model' field in the spec file.")
# !tao model faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt -r /workspace/tao-experiments/faster_rcnn

In [ ]:
print("For Automatic Mixed Precision(AMP) training, please uncomment and run this. Make sure you use the Volta or above GPU arch to enable AMP.")
# !tao model faster_rcnn train --gpu_index $GPU_INDEX --use_amp -e $SPECS_DIR/default_spec_resnet18.txt -r /workspace/tao-experiments/faster_rcnn

 ## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

In [ ]:
!tao model faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt -m /workspace/tao-experiments/faster_rcnn/frcnn_resnet_18.epoch_12.hdf5

 ## 6. Prune trained models <a class="anchor" id="head-6"></a>
 * Specify pre-trained model
 * Equalization criterion
 * Threshold for pruning
 * Output directory to store the model
 
Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is depend on the dataset. A `pth` value below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
!tao model faster_rcnn prune --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_resnet_18.epoch_12.hdf5 \
           -o $USER_EXPERIMENT_DIR/model_1_pruned.hdf5  \
           -eq union  \
           -pth 0.2

In [ ]:
!ls -lht $LOCAL_EXPERIMENT_DIR

 ## 7. Retrain pruned models <a class="anchor" id="head-7"></a>
 * Model needs to be re-trained to bring back accuracy after pruning
 * Specify re-training specification

In [ ]:
# Here we have updated the spec file to include the newly pruned model as a pretrained weights.
!cat $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao model faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt -r /workspace/tao-experiments/faster_rcnn

In [ ]:
# Listing the newly retrained model.
!ls -lht $LOCAL_EXPERIMENT_DIR

 ## 8. Evaluate retrained model <a class="anchor" id="head-8"></a>

In [ ]:
!tao model faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt -m /workspace/tao-experiments/faster_rcnn/frcnn_resnet_18.epoch_12.hdf5

 ## 9. Visualize inferences <a class="anchor" id="head-9"></a>
 In this section, we run the inference tool to generate inferences on the trained models.

In [ ]:
# Copy some test images
!mkdir -p $LOCAL_DATA_DIR/test_samples
!cp $LOCAL_DATA_DIR/testing/image_2/00000* $LOCAL_DATA_DIR/test_samples

In [ ]:
# Running inference for detection on n images
# Please go to $LOCAL_EXPERIMENT_DIR/inference_results_imgs_retrain to see the visualizations.
!tao model faster_rcnn inference --gpu_index $GPU_INDEX \
                -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                -m /workspace/tao-experiments/faster_rcnn/frcnn_resnet_18.epoch_12.hdf5

The `inference` tool produces two outputs. 
1. Overlain images in `$LOCAL_EXPERIMENT_DIR/inference_results_imgs_retrain`
2. Frame by frame bbox labels in kitti format located in `$LOCAL_EXPERIMENT_DIR/inference_dump_labels_retrain`

In [ ]:
# Simple grid visualizer
!pip3 install "matplotlib>=3.3.3, <4.0"
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $LOCAL_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

 ## 10. Deploy! <a class="anchor" id="head-10"></a>

In [ ]:
# Generate .onnx file using tao container
!if [ -f $LOCAL_EXPERIMENT_DIR/frcnn_resnet_18.onnx ]; then rm -f $LOCAL_EXPERIMENT_DIR/frcnn_resnet_18.onnx; fi
!tao model faster_rcnn export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_resnet_18.epoch_12.hdf5  \
                        -o $USER_EXPERIMENT_DIR/frcnn_resnet_18.onnx \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        --target_opset 12 \
                        --gen_ds_config

Using the `tao deploy` container, you can generate a TensorRT engine and verify the correctness of the generated through evaluate and inference. 

The `tao deploy` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please run `tao deploy` command which will instantiate a deploy container, with the exported `.onnx` file on your target device. The `tao deploy` container only works for x86, with discrete NVIDIA GPU's. 

For the jetson devices, please download the tao-converter for jetson from the dev zone link [here](https://developer.nvidia.com/tao-converter). 

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported `.onnx` file along with the calibration cache to the target device and updating the spec file that configures the `gst-nvinfer` element to point to this newly exported model. Usually this file is called `config_infer_primary.txt` for detection models and `config_infer_secondary_*.txt` for classification models.

In [ ]:
# Convert to TensorRT engine (FP32).
!tao deploy faster_rcnn gen_trt_engine --gpu_index $GPU_INDEX \
                        -m $USER_EXPERIMENT_DIR/frcnn_resnet_18.onnx \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        --data_type fp32 \
                        --batch_size 8 \
                        --max_batch_size 4 \
                        --engine_file $USER_EXPERIMENT_DIR/trt.fp32.engine \
                        --results_dir $USER_EXPERIMENT_DIR/

In [ ]:
# Convert to TensorRT engine (FP16).
!tao deploy faster_rcnn gen_trt_engine --gpu_index $GPU_INDEX \
                        -m $USER_EXPERIMENT_DIR/frcnn_resnet_18.onnx \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        --data_type fp16 \
                        --batch_size 8 \
                        --max_batch_size 4 \
                        --engine_file $USER_EXPERIMENT_DIR/trt.fp16.engine \
                        --results_dir $USER_EXPERIMENT_DIR/

In [ ]:
# Convert to TensorRT engine (INT8).
!tao deploy faster_rcnn gen_trt_engine --gpu_index $GPU_INDEX \
                        -m $USER_EXPERIMENT_DIR/frcnn_resnet_18.onnx \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        --data_type int8 \
                        --batch_size 8 \
                        --max_batch_size 4 \
                        --batches 10 \
                        --cal_cache_file $USER_EXPERIMENT_DIR/cal.bin \
                        --cal_image_dir $DATA_DOWNLOAD_DIR/testing/image_2 \
                        --cal_data_file $USER_EXPERIMENT_DIR/cal.tensorfile \
                        --engine_file $USER_EXPERIMENT_DIR/trt.int8.engine \
                        --results_dir $USER_EXPERIMENT_DIR/

In [ ]:
print('Exported model and converted TensorRT engine:')
print('------------')
!ls -lht $LOCAL_EXPERIMENT_DIR

In [ ]:
# Do inference with TensorRT on the generated TensorRT engine
# Please go to $LOCAL_EXPERIMENT_DIR/images_annotated to see the visualizations.
!tao deploy faster_rcnn inference  --gpu_index $GPU_INDEX \
                                   -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                                   -m $USER_EXPERIMENT_DIR/trt.fp32.engine \
                                   -i $DATA_DOWNLOAD_DIR/test_samples \
                                   -r $USER_EXPERIMENT_DIR/trt_infer

The `inference` tool produces two outputs. 
The paths to the two outputs are exactly the same as the first `inference` command.

In [ ]:
# Visualizing the sample images from TensorRT inference.
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $LOCAL_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

In [ ]:
import os
import shutil
val_split = 0.14  # Val split from the spec file
img_ext = '.png'
lab_ext = '.txt'

# Get images and labels from the partitioned validation set
images_root = f"{os.environ['LOCAL_DATA_DIR']}/training/image_2"
labels_root = f"{os.environ['LOCAL_DATA_DIR']}/training/label_2"
images_list = [os.path.splitext(imfile)[0] for imfile in sorted(os.listdir(images_root)) if imfile.endswith(img_ext)]
num_val_images = int(len(images_list) * val_split)

# Copy the data to a separate directory for evaluation
os.makedirs(os.path.join(f"{os.environ['LOCAL_DATA_DIR']}/training", "images_val"), exist_ok=True)
os.makedirs(os.path.join(f"{os.environ['LOCAL_DATA_DIR']}/training", "labels_val"), exist_ok=True)

for fname in images_list[:num_val_images]:
    shutil.copyfile(os.path.join(images_root, fname + img_ext), \
                             os.path.join(f"{os.environ['LOCAL_DATA_DIR']}/training", "images_val", fname + img_ext))
    shutil.copyfile(os.path.join(labels_root, fname + lab_ext), \
                             os.path.join(f"{os.environ['LOCAL_DATA_DIR']}/training", "labels_val", fname + lab_ext))

In [ ]:
# Doing evaluation with the generated TensorRT engine
!tao deploy faster_rcnn evaluate --gpu_index $GPU_INDEX \
                                 -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                                 -m $USER_EXPERIMENT_DIR/trt.int8.engine \
                                 -i $DATA_DOWNLOAD_DIR/training/images_val \
                                 -l $DATA_DOWNLOAD_DIR/training/labels_val \
                                 -r $USER_EXPERIMENT_DIR/

 ## 11. QAT workflow <a class="anchor" id="head-11"></a>

In this section, we will explore the typical Quantization-Aware Training(QAT) workflow with TAO. QAT workflow is almost the same as non-QAT workflow except for two major differences:
1. set `enable_qat` to `True` in training and retraining spec files to enable the QAT for training/retraining
2. when doing export in INT8 mode, the calibration cache is extracted directly from the QAT .hdf5 model, so no need to specify any TensorRT INT8 calibration related arguments for `export`

 ### 11.1. Training <a class="anchor" id="head-10.1"></a>

In [ ]:
# set enable_qat to True in training spec file to enable QAT training
!sed -i 's/enable_qat: False/enable_qat: True/' $LOCAL_SPECS_DIR/default_spec_resnet18.txt
!cat $LOCAL_SPECS_DIR/default_spec_resnet18.txt

In [ ]:
# run QAT training
!tao model faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt -r /workspace/tao-experiments/faster_rcnn

 ### 11.2. Evaluation <a class="anchor" id="head-10.2"></a>

In [ ]:
!tao model faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt -m /workspace/tao-experiments/faster_rcnn/frcnn_resnet_18.epoch_12.hdf5

 ### 11.3. Pruning <a class="anchor" id="head-10.3"></a>

In [ ]:
!tao model faster_rcnn prune --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_resnet_18.epoch_12.hdf5 \
           -o $USER_EXPERIMENT_DIR/model_1_pruned.hdf5  \
           -eq union  \
           -pth 0.2

 ### 11.4. Retraining <a class="anchor" id="head-10.4"></a>

In [ ]:
# set enable_qat to True in retraining spec file to enable QAT
!sed -i 's/enable_qat: False/enable_qat: True/' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!cat $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
!tao model faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt -r /workspace/tao-experiments/faster_rcnn

 ### 11.5. Evaluation of the retrained model <a class="anchor" id="head-10.5"></a>

In [ ]:
# do evaluation with .hdf5 model
!tao model faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt -m /workspace/tao-experiments/faster_rcnn/frcnn_resnet_18.epoch_12.hdf5

 ### 11.6. Inference of the retrained model <a class="anchor" id="head-10.6"></a>

In [ ]:
# do inference with .hdf5 model
!tao model faster_rcnn inference --gpu_index $GPU_INDEX \
                -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                -m /workspace/tao-experiments/faster_rcnn/frcnn_resnet_18.epoch_12.hdf5

In [ ]:
# Visualizing the sample images
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

 ### 11.7. Deployment of the QAT model <a class="anchor" id="head-10.7"></a>

In [ ]:
# Calibration JSON file is required for INT8 engine generation from tao deploy
!if [ -f $LOCAL_EXPERIMENT_DIR/frcnn_resnet_18_int8_qat.onnx ]; then rm -f $LOCAL_EXPERIMENT_DIR/frcnn_resnet_18_int8_qat.onnx; fi
!if [ -f $LOCAL_EXPERIMENT_DIR/cal.bin ]; then rm -f $LOCAL_EXPERIMENT_DIR/cal.bin; fi
!tao model faster_rcnn export --gpu_index $GPU_INDEX \
                        -m $USER_EXPERIMENT_DIR/frcnn_resnet_18.epoch_12.hdf5  \
                        -o $USER_EXPERIMENT_DIR/frcnn_resnet_18_int8_qat.onnx \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        --cal_json_file $USER_EXPERIMENT_DIR/cal.json \
                        --target_opset 12 \
                        --gen_ds_config

In [ ]:
# Convert to TensorRT engine(INT8).
# No need for calibration dataset for QAT model INT8 export
!tao deploy faster_rcnn gen_trt_engine --gpu_index $GPU_INDEX \
                        -m $USER_EXPERIMENT_DIR/frcnn_resnet_18_int8_qat.onnx \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        --data_type int8 \
                        --cal_cache_file $USER_EXPERIMENT_DIR/cal.bin \
                        --cal_json_file $USER_EXPERIMENT_DIR/cal.json \
                        --batch_size 8 \
                        --max_batch_size 4 \
                        --engine_file $USER_EXPERIMENT_DIR/trt.int8.qat.engine \
                        --results_dir $USER_EXPERIMENT_DIR/

In [ ]:
print('Exported model and converted TensorRT engine:')
print('------------')
!ls -lht $LOCAL_EXPERIMENT_DIR

In [ ]:
# Do inference with TensorRT on the generated TensorRT engine
# Please go to $LOCAL_EXPERIMENT_DIR/images_annotated to see the visualizations.
!tao deploy faster_rcnn inference --gpu_index $GPU_INDEX \
                                  -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                                  -m $USER_EXPERIMENT_DIR/trt.int8.qat.engine \
                                  -i $DATA_DOWNLOAD_DIR/test_samples \
                                  -r $USER_EXPERIMENT_DIR/trt_qat_infer

In [ ]:
# Visualizing the sample images from TensorRT inference.
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

In [ ]:
# Doing evaluation with the generated TensorRT engine
!tao deploy faster_rcnn evaluate --gpu_index $GPU_INDEX \
                                 -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                                 -m $USER_EXPERIMENT_DIR/trt.int8.qat.engine \
                                 -i $DATA_DOWNLOAD_DIR/training/images_val \
                                 -l $DATA_DOWNLOAD_DIR/training/labels_val \
                                 -r $USER_EXPERIMENT_DIR/